# PHASE 1 PROJECT - Aviation Risk Analysis & Recommendations

## Introduction
As the company is seeking to diversify its portfolio through expansion into commercial and private aviation, it is impotrant to have a deep understanding of operational risks. Using the accident data from the National Transportation Safety Board, this project evaluates the safety records of various aircraft manufacturesrs and models from 1962 to 2023. I will explore the data to determine the lowest risk options available. This will inform actionable recommendations to stakeholders to acquire safe and reliable fleet.

## Business Understanding
1. Problem Statement
The primary challenge is that the company lacks internal expertise regarding the safety profiles and potential risks associated with different types of aircraft. Investing in high-risk aircraft could lead to financial loss, legal liability, reputational damage, and loss of life. The company needs to know which aircraft present the lowest risk to ensure the safety of operations and the viability of the new business unit.

2. Main Objective
The goal of this analysis is to determine which aircraft manufacturers and models have the lowest risk profile to support the company's new business endeavor.

- Analyzing historical aviation accident data to identify trends in safety.
- Evaluating risk based on key factors such as aircraft make, engine type, and phase of flight.
- Providing three concrete, actionable recommendations to the head of the new aviation division to guide their purchasing decisions.

4. Stakeholders
Primary Stakeholder: Head of the new Aviation Division.
Secondary Stakeholders: Executive Board, Investors, and potential future passengers/clients who rely on the company's commitment to safety.

5. Data Source
The analysis utilizes the National Transportation Safety Board (NTSB) Aviation Accident Dataset, which contains information on civil aviation accidents and selected incidents within the United States and international waters from 1962 to 2023. This comprehensive dataset allows for a robust assessment of long-term safety trends and specific incident causes.

---

## Data Understanding
#### Data source
The dataset for this analysis is sourved from **National Transportation Safety Board, that covers aviation accidents and incidents involving aircrafts in the United States and international waters from 1962 to 2023.
This data serves as a reliable sourve to determine aircraft safety records.

#### Data Schema
The dataset includes tens of thousands of records, where each row represents a single aviation accident or incident.
Each record contains information about:
- The aircraft (type, category, manufacturer, model)
- The event (date, location, purpose of flight)
- The environment (weather, light conditions)
- The outcome (injuries, fatalities, damage level)
This makes the dataset comprehensive enough to evaluate both accident frequency and accident severity across aircraft types.

#### Data Quality
##### Missing Values
- Some older records (1960s–1980s) may lack detail
- Certain columns may have missing values (None) that may need to be filled or droped.
- Weather and light condition fields frequently contain “Unknown”

##### Inconsistent Values
Since the data set is based on a large period, certain values may have format changes or spelling errors.
- Manufacturers may appear in multiple forms
- Aircraft models may use different formatting or spacing

##### Outliers
- Extremely old or rare aircraft types
- Occasional data-entry errors
- Records with zero injuries but aircraft recorded as “Destroyed”

---